# Predict

In [2]:
import numpy as np
import random
import os
import math
from itertools import product

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from torch.utils.tensorboard import SummaryWriter

from torch.autograd import Variable

import torchvision.models as models
from torchvision import transforms

import albumentations as A
import albumentations.pytorch

In [89]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [7]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':50,
    'PATIENCE':10,
    'class':14
}

In [13]:
Train_path = '/home/lab17/jupyter_home/Data/product_image/Training/'
Valid_path = '/home/lab17/jupyter_home/Data/product_image/Validation/'
Test_path = '/home/lab17/jupyter_home/git/test_img/'
save_graph_path = './result/'
save_model_path = '/home/lab17/jupyter_home/saved_models/'

# Load Data

In [46]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = os.path.join(data_dir, 'dessert')
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
#             get image path
#             img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_path_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name[:5])
            
            # get label
            label_list.append(''.join(label))
                
#     return img_path_list, label_list
    return label_list

In [50]:
label_list = get_train_data(Train_path)

In [56]:
# encoder
le = preprocessing.LabelEncoder()
targets = le.fit_transform(label_list)
print('--targets\n' , targets)

label_encoder = {key:val for key, val in zip(label_list, targets)}
print(label_encoder)

--targets
 [12 10  2  8  0  3 11  9  4  7 13  5  6  1]
{'55701': 12, '45661': 10, '35211': 2, '45659': 8, '25222': 0, '35584': 3, '55034': 11, '45660': 9, '35585': 4, '45658': 7, '55702': 13, '45030': 5, '45657': 6, '25228': 1}


In [38]:
def get_test_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = data_dir
    
#     for product in os.listdir(image_path):

    # get image path
    img_path_list.extend(glob(os.path.join(image_path, '*.jpg')))
    img_path_list.extend(glob(os.path.join(image_path, '*.png')))
    label_list = [ip[len('/home/lab17/jupyter_home/git/test_img/'):-6] for ip in img_path_list]

    # get label
#     label_list.append(''.join(label))
                
    return img_path_list, label_list

In [39]:
test_img_path, test_label_list = get_test_data(Test_path)

In [59]:
# test data
test_x = test_img_path
# 레이블을 one-hot-vector로 변환
test_y = [label_encoder[key] for key in test_label_list]
test_targets = torch.as_tensor(test_y)
one_hot_test_y = F.one_hot(test_targets)

In [60]:
print([f'{i}, {y}' for i, y in zip(test_label_list, test_y)])

['55034, 11', '45661, 10', '25228, 1', '25222, 0', '45659, 8', '55701, 12', '45030, 5', '35211, 2', '45660, 9', '45659, 8', '35585, 4', '55702, 13', '25222, 0', '55702, 13', '35211, 2', '45030, 5', '45661, 10', '35584, 3', '25222, 0', '55701, 12', '35211, 2', '35211, 2', '55701, 12', '35584, 3', '45660, 9', '55702, 13', '25222, 0', '45657, 6', '45657, 6', '55701, 12', '45657, 6']


In [104]:
class AlbumentationsCustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        
        # By default OpenCV uses BGR color space for color images,
        # so we need to convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.train_mode:
#             image = image.astype(np.int16)
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
        else:
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
    
    def __len__(self):
        return len(self.img_path_list)
    
A_test_transform = albumentations.Compose([
                                    A.Resize(256, 256),
                                    A.Normalize(mean=(0.744859, 0.735139, 0.711357), std=(0.100712, 0.120692, 0.167998)),  
#                                     A.pytorch.transforms.ToTensor(),
                                    A.pytorch.transforms.ToTensorV2(transpose_mask=True),
                                ])

A_test_dataset = AlbumentationsCustomDataset(test_x, one_hot_test_y, train_mode=False, transforms=A_test_transform)
A_test_loader = DataLoader(A_test_dataset, batch_size = 4, shuffle=False, num_workers=0, collate_fn=None)

# Inference

In [105]:
pred_ensemble = []
batch_size = 34

#---------
model_name = 'ResNet50'
model_lr = '0.0001'
model_optim = 'adam'
model_sch = 'CosineAnnealing' 
#---------

model_test = ResNet50().to(device)
# model_path = '/home/lab17/jupyter_home/saved_models/{}_{}_{}_{}_example.pth'.format(model_name, model_lr, model_optim, model_sch)
# model_test.load_state_dict(torch.load(model_path))
# model_test.load_state_dict(torch.load('/home/lab17/jupyter_home/saved_models/{}_{}_{}_{}_example.pth'.format(model_name, model_lr, model_optim, model_sch))['state_dict'])
model_test.load_state_dict(torch.load('/home/lab17/jupyter_home/saved_models/{}_{}_{}_{}_example.pth'.format(model_name, model_lr, model_optim, model_sch)))
model_test.eval()

criterion = torch.nn.CrossEntropyLoss()
test_loss = []

for img, label in tqdm(iter(A_test_loader)):
    img, label = img.float().to(device), label.float().to(device)
    
    # Data -> Model -> Output
    logit = model_test(img)
    logit = torch.squeeze(logit)
    print(logit)
    
    # Calc loss
    loss = criterion(logit, label)

    test_loss.append(loss.item())
    
print(np.mean(test_loss))





# f_pred = []
# pred_prob = []

# image_data = Image.open('home/lab17/jupyter_home/Data/product_test/img.jpg')

# image_transform = transforms.Compose([
#     transforms.Resize(size=256),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.744859, 0.735139, 0.711357],
#                          std=[0.100712, 0.120692, 0.167998])
# ])

# x = image_transform(image_data)
# pred = model_test(x)
# pred_prob.extend(pred.detach().cpu().numpy())
# f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

# label_decoder = {val:key for key, val in zip(range(CFG['class']), sorted(label_list))}

# f_result = [label_decoder[result] for result in f_pred]

# print(f_result)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.76 GiB total capacity; 2.61 GiB already allocated; 24.75 MiB free; 2.64 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF